<a href="https://colab.research.google.com/github/Wallace-NMelo/Fotografia-Computacional/blob/main/Sistemas_Heterogeneos/Avalia%C3%A7%C3%A3o_03_An%C3%A1lise_e_avalia%C3%A7%C3%A3o_de_performance_em_sistemas_heterog%C3%AAneos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre-Setup**

In [ ]:
!nvidia-smi

Sun May 19 13:00:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.29
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-88mh461a
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-88mh461a
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-dmv1by8t/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%reload_ext plugin

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpvgx8qv6j".


In [ ]:
%%cuda
#include <iostream>
    int
    main()
{
    std::cout << "C++ test\n";
    return 0;
}

C++ test



In [ ]:
# Download and install CUDA 12.1
! set -x \
&& cd $(mktemp -d) \
&& wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run \
&& sudo sh cuda_12.1.0_530.30.02_linux.run --silent --toolkit \
&& rm cuda_12.1.0_530.30.02_linux.run

++ mktemp -d
+ cd /tmp/tmp.d2JKlzYzzc
+ wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
--2024-05-19 13:39:05--  https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4245586997 (4.0G) [application/octet-stream]
Saving to: ‘cuda_12.1.0_530.30.02_linux.run’

cuda_12.1.0_530.30. 100%[===================>]   3.95G   136MB/s    in 24s     

2024-05-19 13:39:29 (167 MB/s) - ‘cuda_12.1.0_530.30.02_linux.run’ saved [4245586997/4245586997]

+ sudo sh cuda_12.1.0_530.30.02_linux.run --silent --toolkit
+ rm cuda_12.1.0_530.30.02_linux.run


# **Matriz Transposta**

O operador de matriz transposta (T) é definido como:

Para uma matriz A (m x n), sua transposição T(A) é definida pela matriz At (n x m), onde a diagonal principal de At é a reflexão da diagonal principal de A, e, as linhas e colunas de At correspondem, respectivamente, as colunas e linhas de A.



In [ ]:
!apt install build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### Escreva uma função T_seq que realiza a transposição de uma matriz MxN utilizando apenas o host para execução

In [ ]:
%%file t_seq.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void transpose(float *A, float *At, int m, int n)
{
    for(int i = 0; i < n; i++)
    {
        for(int j = 0; j < m; j++)
        {
            At[i * m + j] = A[j * n + i];
        }
    }
}

int main()
{
    srand(time(NULL));
    int m = 5, n = 10;

    float *A = (float *) malloc(m * n * sizeof(float));
    float *At = (float *) malloc(n * m * sizeof(float));

    for (int i = 0; i < m * n; i++)
    {
        A[i] = (float) rand() / (float) RAND_MAX;
    }

    for(int i = 0; i < m; i++)
    {
        for(int j = 0; j < n; j++)
        {
            printf("%.2f ", A[i * n + j]);
        }
        printf("\n");
    }

    transpose(A, At, m, n);
    printf("\n");

    for(int i = 0; i < n; i++)
    {
        for(int j = 0; j < m; j++)
        {
            printf("%.2f ", At[i * m + j]);
        }
        printf("\n");
    }

    free(A);
    free(At);

    return 0;
}

Writing t_seq.cu


In [ ]:
!nvcc -o t_seq t_seq.cu

In [ ]:
%%time

!./t_seq

0.55 0.67 0.81 0.76 0.63 0.43 0.04 0.32 0.33 0.56 
0.79 0.43 0.27 0.29 0.94 0.62 0.01 1.00 0.02 0.70 
0.49 0.97 0.04 0.64 0.75 0.78 0.59 0.34 0.58 0.74 
0.35 0.13 0.41 0.16 0.89 0.04 0.59 0.93 0.36 0.92 
0.49 0.14 0.36 0.75 0.43 0.29 0.38 0.44 0.29 0.40 

0.55 0.79 0.49 0.35 0.49 
0.67 0.43 0.97 0.13 0.14 
0.81 0.27 0.04 0.41 0.36 
0.76 0.29 0.64 0.16 0.75 
0.63 0.94 0.75 0.89 0.43 
0.43 0.62 0.78 0.04 0.29 
0.04 0.01 0.59 0.59 0.38 
0.32 1.00 0.34 0.93 0.44 
0.33 0.02 0.58 0.36 0.29 
0.56 0.70 0.74 0.92 0.40 
CPU times: user 4.33 ms, sys: 1.16 ms, total: 5.5 ms
Wall time: 106 ms


### Defina o kernel T_gpu que realiza a transposição de matriz (MxN) utilizando o device. Escreva o kernel de forma que o mapeamento de threads seja 1:1 para cada posição da matriz de saída

In [ ]:
# %%cuda
# %%nvprof
%%file t_gpu.cu
#include <stdio.h>

__global__ void transpose(int *A, int *At, int rows, int cols) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < cols && y < rows) {
        At[x * rows + y] = A[y * cols + x];
    }
}
int main() {
    int rows = 10;
    int cols = 5;

    int *h_input, *h_output;
    int *d_input, *d_output;
    int size = rows * cols * sizeof(int);

    // Allocate memory on host
    h_input = (int*)malloc(size);
    h_output = (int*)malloc(size);

    // Initialize input matrix on host
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_input[i * cols + j] = i * cols + j;
        }
    }

    // Allocate memory on device
    cudaMalloc((void**)&d_input, size);
    cudaMalloc((void**)&d_output, size);

    // Copy input matrix from host to device
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((cols + blockDim.x - 1) / blockDim.x, (rows + blockDim.y - 1) / blockDim.y);

    // Call kernel function
    transpose<<<gridDim, blockDim>>>(d_input, d_output, rows, cols);

    // Copy output matrix from device to host
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    // Print output matrix
    printf("Input Matrix:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%d ", h_input[i * cols + j]);
        }
        printf("\n");
    }

    printf("\nTransposed Matrix:\n");
    for (int i = 0; i < cols; i++) {
        for (int j = 0; j < rows; j++) {
            printf("%d ", h_output[i * rows + j]);
        }
        printf("\n");
    }

    // Free memory
    free(h_input);
    free(h_output);
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Writing t_gpu.cu


In [ ]:
!nvcc -o t_gpu t_gpu.cu

In [ ]:
!./t_gpu

Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 


In [ ]:
!apt-get install -y nsight-compute-2024.1.1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  nsight-compute-2024.1.1
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 594 MB of archives.
After this operation, 1,427 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  nsight-compute-2024.1.1 2024.1.1.4-1 [594 MB]
Fetched 594 MB in 8s (77.3 MB/s)
Selecting previously unselected package nsight-compute-2024.1.1.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../nsight-compute-2024.1.1_2024.1.1.4-1_amd64.deb ...
Unpacking nsight-compute-2024.1.1 (2024.1.1.4-1) ...
Setting up nsight-compute-2024.1.1 (2024.1.1.4-1) ...


### Qual o valor da taxa de operações por acesso à memória do kernel T_gpu?

### **1.0 Flops/Byte**



In [ ]:
!ncu --target-processes all ./t_gpu

==PROF== Connected to process 22987 (/content/t_gpu)
==PROF== Profiling "transpose" - 0: 0%....50%....100% - 9 passes
Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 
==PROF== Disconnected from process 22987
[22987] t_gpu@127.0.0.1
  transpose(int *, int *, int, int) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.68
    SM Frequency            cycle/usecond       575.59
    Elapsed Cycles                  cycle        2,155
    Memory Through

In [ ]:
!nvprof ./t_gpu

==2952== NVPROF is profiling process 2952, command: ./t_gpu
Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 
==2952== Profiling application: ./t_gpu
==2952== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.00%  3.2960us         1  3.2960us  3.2960us  3.2960us  transpose(int*, int*, int, int)
                   32.04%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   17.96%  1.1840us         1  1.1840us  1.1840us  1.1840us  [CUDA memcpy HtoD]
      API calls:   99.41%  111.05ms         2  55.525ms  12.707us  111.04ms  cudaMalloc
                    0.21%  235.75us         1  235.75us  235.75us  235.75

Execute o profile (visual ou CLI) e obtenha as medidas de execução do kernel (tempo, memória, ocupância).

### Considerando as otimizações de código que foram vistas na disciplina. Como você melhoraria a execução deste kernel?




O código de transposição de matriz pode ser otimizado de várias maneiras para melhorar o desempenho, especialmente em um ambiente CUDA. Aqui estão algumas otimizações que podem ser aplicadas:

1. Uso de Memória Compartilhada (Shared Memory):
A memória compartilhada é muito mais rápida do que a memória global. Utilizá-la para armazenar blocos de dados que são acessados frequentemente pode reduzir o tempo de acesso à memória global.


2. Uso de Barramentos (Thread Synchronization):
A sincronização das threads dentro de um bloco garante que todas as operações de leitura e escrita na memória compartilhada sejam concluídas antes de prosseguir.


###Escreva um kernel T_opt aplicando as melhorias que você sugeriu anteriormente

In [ ]:
%%file t_opt.cu
#include <stdio.h>

#define TILE_DIM 16  // Define the size of the tile/block

__global__ void transpose_optimized(int *input, int *output, int rows, int cols) {
    __shared__ int tile[TILE_DIM][TILE_DIM + 1]; // Avoid bank conflicts

    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;

    if (x < cols && y < rows) {
        tile[threadIdx.y][threadIdx.x] = input[y * cols + x];
    }
    __syncthreads();

    x = blockIdx.y * TILE_DIM + threadIdx.x; // Transpose block offset
    y = blockIdx.x * TILE_DIM + threadIdx.y;

    if (x < rows && y < cols) {
        output[y * rows + x] = tile[threadIdx.x][threadIdx.y];
    }
}

int main() {
    int rows = 10;
    int cols = 5;

    int *h_input, *h_output;
    int *d_input, *d_output;
    int size = rows * cols * sizeof(int);

    // Allocate memory on host
    h_input = (int*)malloc(size);
    h_output = (int*)malloc(size);

    // Initialize input matrix on host
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_input[i * cols + j] = i * cols + j;
        }
    }

    // Allocate memory on device
    cudaMalloc((void**)&d_input, size);
    cudaMalloc((void**)&d_output, size);

    // Copy input matrix from host to device
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(TILE_DIM, TILE_DIM);
    dim3 gridDim((cols + TILE_DIM - 1) / TILE_DIM, (rows + TILE_DIM - 1) / TILE_DIM);

    // Call kernel function
    transpose_optimized<<<gridDim, blockDim>>>(d_input, d_output, rows, cols);

    // Copy output matrix from device to host
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    // Print output matrix
    printf("Input Matrix:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%d ", h_input[i * cols + j]);
        }
        printf("\n");
    }

    printf("\nTransposed Matrix:\n");
    for (int i = 0; i < cols; i++) {
        for (int j = 0; j < rows; j++) {
            printf("%d ", h_output[i * rows + j]);
        }
        printf("\n");
    }

    // Free memory
    free(h_input);
    free(h_output);
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}



Writing t_opt.cu


In [ ]:
!nvcc -o t_opt t_opt.cu

In [ ]:
!ncu ./t_opt

==PROF== Connected to process 10957 (/content/t_opt)
==PROF== Profiling "transpose_optimized" - 0: 0%....50%....100% - 9 passes
Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 
==PROF== Disconnected from process 10957
[10957] t_opt@127.0.0.1
  transpose_optimized(int *, int *, int, int) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.70
    SM Frequency            cycle/usecond       574.38
    Elapsed Cycles                  cycle        2,20

In [ ]:
!nvprof ./t_opt

==3010== NVPROF is profiling process 3010, command: ./t_opt
Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 
==3010== Profiling application: ./t_opt
==3010== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.96%  3.3920us         1  3.3920us  3.3920us  3.3920us  transpose_optimized(int*, int*, int, int)
                   31.25%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
                   17.79%  1.1840us         1  1.1840us  1.1840us  1.1840us  [CUDA memcpy HtoD]
      API calls:   65.53%  120.01ms         2  60.006ms  10.513us  120.00ms  cudaMalloc
                   34.22%  62.661ms         1  62.661ms  62.661

In [ ]:
!ncu ./t_opt

==PROF== Connected to process 23671 (/content/t_opt)
==PROF== Profiling "transpose_optimized" - 0: 0%....50%....100% - 9 passes
Input Matrix:
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 
40 41 42 43 44 
45 46 47 48 49 

Transposed Matrix:
0 5 10 15 20 25 30 35 40 45 
1 6 11 16 21 26 31 36 41 46 
2 7 12 17 22 27 32 37 42 47 
3 8 13 18 23 28 33 38 43 48 
4 9 14 19 24 29 34 39 44 49 
==PROF== Disconnected from process 23671
[23671] t_opt@127.0.0.1
  transpose_optimized(int *, int *, int, int) (1, 1, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.73
    SM Frequency            cycle/usecond       576.41
    Elapsed Cycles                  cycle        2,21

### Qual o valor da taxa de operações por acesso à memória do kernel T_opt?

### **1.0 Flops/Byte**

# Bubblesort
###Bubblesort é um algoritmo utilizado para ordenação de itens em um array, o processo é realizado comparando-se os elementos do array dois a dois
https://pt.wikipedia.org/wiki/Bubble_sort


procedure bubbleSort( A : lista de itens ordenaveis ) defined as:
```  
do
    trocado := false
    for each i in 0 to length( A ) - 2 do:
      // verificar se os elementos estão na ordem certa
      if A[ i ] > A[ i + 1 ] then
        // trocar elementos de lugar
        trocar( A[ i ], A[ i + 1 ] )
        trocado := true
      end if
    end for
  // enquanto houver elementos sendo reordenados.
  while trocado
end procedure
```
Com base no algoritmo de bubblesort, responda as questões abaixo:

### Considerando que temos um vetor de no máximo 1024 elementos (N=[1, 1024]). Defina um kernel que utiliza apenas 1 bloco para realizar a ordenação deste vetor.

In [ ]:
%%file bubble_sort.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

__device__ int mutex = 0;

__device__ void lock(int *mutex)
{
    while (atomicCAS(mutex, 0, 1) != 0);
}

__device__ void unlock(int *mutex)
{
    atomicExch(mutex, 0);
}

__device__ void swap(float *A, int i, int j)
{
    float t = A[i];
    A[i] = A[j];
    A[j] = t;
}

__global__ void bubble_sort(float *A, int n)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < n)
    {
        for (int i = 0; i < n - 1; i++)
        {
            for (int j = 0; j < n - i - 1; j++)
            {
                lock(&mutex);
                if (A[j] > A[j + 1]) swap(A, j, j + 1);
                unlock(&mutex);
            }
        }
    }
}

int main()
{
    srand(time(NULL));
    int n = 10;

    float *A_h, *A_d;

    A_h = (float *) malloc(n * sizeof(float));
    cudaMalloc(&A_d, n * sizeof(float));

    for (int i = 0; i < n; i++)
    {
        A_h[i] = (float) rand() / (float) RAND_MAX;
    }

    for (int i = 0; i < n; i++)
    {
        printf("%.2f ", A_h[i]);
    }
    printf("\n");

    cudaMemcpy(A_d, A_h, n * sizeof(float), cudaMemcpyHostToDevice);
    bubble_sort<<<1, 1>>>(A_d, n);
    cudaMemcpy(A_h, A_d, n * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i < n; i++)
    {
        printf("%.2f ", A_h[i]);
    }
    printf("\n");

    free(A_h);
    cudaFree(A_d);

    return 0;
}

Writing bubble_sort.cu


In [ ]:
!nvcc -o bubble_sort bubble_sort.cu

In [ ]:
%%time
!./bubble_sort

0.64 0.09 0.04 0.84 0.11 0.19 0.41 0.86 0.16 0.77 
0.04 0.09 0.11 0.16 0.19 0.41 0.64 0.77 0.84 0.86 
CPU times: user 4.89 ms, sys: 828 µs, total: 5.71 ms
Wall time: 206 ms


In [ ]:
!ncu bubble_sort

==PROF== Connected to process 16389 (/content/bubble_sort)
0.22 0.84 0.46 0.81 0.91 0.86 0.50 0.25 0.08 0.53 
==PROF== Profiling "bubble_sort(float *, int)" - 0: 0%....50%....100% - 9 passes
0.08 0.22 0.25 0.46 0.50 0.53 0.81 0.84 0.86 0.91 
==PROF== Disconnected from process 16389
[16389] bubble_sort@127.0.0.1
  bubble_sort(float *, int) (1, 1, 1)x(1, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.96
    SM Frequency            cycle/usecond       583.98
    Elapsed Cycles                  cycle       16,708
    Memory Throughput                   %         0.09
    DRAM Throughput                     %         0.04
    Duration                      usecond        28.61
    L1/TEX Cache Throughput             %         3.51
    L2 

### Como você ajustaria o seu kernel para entradas de valores arbitrários?
Implemente a versão customizada que pode executar a ordenação em múltiplos blocos

In [ ]:
%%writefile bubble_sort_2.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

__device__ int mutex = 0;

__device__ void lock(int *mutex)
{
    while (atomicCAS(mutex, 0, 1) != 0);
}

__device__ void unlock(int *mutex)
{
    atomicExch(mutex, 0);
}

__device__ void swap(float *A, int i, int j)
{
    float t = A[i];
    A[i] = A[j];
    A[j] = t;
}

__global__ void bubble_sort(float *A, int n)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < n)
    {
        for (int i = 0; i < n - 1; i++)
        {
            for (int j = 0; j < n - i - 1; j++)
            {
                lock(&mutex);
                if (A[j] > A[j + 1]) swap(A, j, j + 1);
                unlock(&mutex);
            }
        }
    }
}

int main()
{
    srand(time(NULL));
    int n = 10;

    float *A_h, *A_d;

    A_h = (float *) malloc(n * sizeof(float));
    cudaMalloc(&A_d, n * sizeof(float));

    for (int i = 0; i < n; i++)
    {
        A_h[i] = (float) rand() / (float) RAND_MAX;
    }

    for (int i = 0; i < n; i++)
    {
        printf("%.2f ", A_h[i]);
    }
    printf("\n");

    cudaMemcpy(A_d, A_h, n * sizeof(float), cudaMemcpyHostToDevice);
    bubble_sort<<<n, 1>>>(A_d, n);
    cudaMemcpy(A_h, A_d, n * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i < n; i++)
    {
        printf("%.2f ", A_h[i]);
    }
    printf("\n");

    free(A_h);
    cudaFree(A_d);

    return 0;
}

Writing bubble_sort_2.cu


In [ ]:
!nvcc -o bubble_sort_2 bubble_sort_2.cu

In [ ]:
%%time
!./bubble_sort_2

0.79 0.61 0.46 0.78 0.35 0.38 0.47 0.65 0.56 0.49 
0.35 0.38 0.46 0.47 0.49 0.61 0.65 0.78 0.79 0.79 
CPU times: user 3.88 ms, sys: 1.95 ms, total: 5.83 ms
Wall time: 205 ms


In [ ]:
!ncu bubble_sort_2

==PROF== Connected to process 17961 (/content/bubble_sort_2)
0.00 0.93 0.56 0.92 0.72 0.82 0.19 0.23 0.67 0.15 
==PROF== Profiling "bubble_sort(float *, int)" - 0: 0%....50%....100% - 9 passes
0.00 0.19 0.23 0.56 0.72 0.67 0.72 0.82 0.93 0.93 
==PROF== Disconnected from process 17961
[17961] bubble_sort_2@127.0.0.1
  bubble_sort(float *, int) (10, 1, 1)x(1, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         5.01
    SM Frequency            cycle/usecond       584.80
    Elapsed Cycles                  cycle      132,924
    Memory Throughput                   %         0.28
    DRAM Throughput                     %         0.01
    Duration                      usecond       227.30
    L1/TEX Cache Throughput             %         1.20
  